In [ ]:
!pip install pandas
!pip install numpy
!pip install requests
!pip install sqlalchemy

In [ ]:
import pandas as pd
import requests
import sqlite3

In [ ]:
#Aqui vamos criar nosso banco de dados sqlite e dar o nome ao db || Vamos criar o cursor tb

con = sqlite3.connect('paises.db')
cur = con.cursor()

In [ ]:
#Aqui vamos puxar os dados da API pública 'restcountries' e extrair com requests

api_restcountries = ("https://restcountries.com/v3.1/all")

response = requests.get(api_restcountries)

if response.status_code == 200:
  print("Importação com sucesso!")
  data = response.json()

  df = pd.DataFrame(data)

elif response.status_code == 400:
  print("Falha na importação!")




Importação com sucesso!


In [158]:
df.head() #nem abre isso aqui, é uma bagunça

,name,tld,cca2,ccn3,cca3,independent,status,unMember,currencies,idd,...,flags,coatOfArms,startOfWeek,capitalInfo,cioc,subregion,fifa,borders,gini,postalCode
0,"{'common': 'South Georgia', 'official': 'South...",[.gs],GS,239,SGS,False,officially-assigned,False,"{'SHP': {'name': 'Saint Helena pound', 'symbol...","{'root': '+5', 'suffixes': ['00']}",...,"{'png': 'https://flagcdn.com/w320/gs.png', 'sv...",{},monday,"{'latlng': [-54.28, -36.5]}",NaN,NaN,NaN,NaN,NaN,NaN
1,"{'common': 'Grenada', 'official': 'Grenada', '...",[.gd],GD,308,GRD,True,officially-assigned,True,"{'XCD': {'name': 'Eastern Caribbean dollar', '...","{'root': '+1', 'suffixes': ['473']}",...,"{'png': 'https://flagcdn.com/w320/gd.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [32.38, -64.68]}",GRN,Caribbean,GRN,NaN,NaN,NaN
2,"{'common': 'Switzerland', 'official': 'Swiss C...",[.ch],CH,756,CHE,True,officially-assigned,True,"{'CHF': {'name': 'Swiss franc', 'symbol': 'Fr.'}}","{'root': '+4', 'suffixes': ['1']}",...,"{'png': 'https://flagcdn.com/w320/ch.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [46.92, 7.47]}",SUI,Western Europe,SUI,"[AUT, FRA, ITA, LIE, DEU]",{'2018': 33.1},"{'format': '####', 'regex': '^(\d{4})$'}"
3,"{'common': 'Sierra Leone', 'official': 'Republ...",[.sl],SL,694,SLE,True,officially-assigned,True,"{'SLL': {'name': 'Sierra Leonean leone', 'symb...","{'root': '+2', 'suffixes': ['32']}",...,"{'png': 'https://flagcdn.com/w320/sl.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [8.48, -13.23]}",SLE,Western Africa,SLE,"[GIN, LBR]",{'2018': 35.7},NaN
4,"{'common': 'Hungary', 'official': 'Hungary', '...",[.hu],HU,348,HUN,True,officially-assigned,True,"{'HUF': {'name': 'Hungarian forint', 'symbol':...","{'root': '+3', 'suffixes': ['6']}",...,"{'png': 'https://flagcdn.com/w320/hu.png', 'sv...",{'png': 'https://mainfacts.com/media/images/co...,monday,"{'latlng': [47.5, 19.08]}",HUN,Central Europe,HUN,"[AUT, HRV, ROU, SRB, SVK, SVN, UKR]",{'2018': 29.6},"{'format': '####', 'regex': '^(\d{4})$'}"


In [159]:
#Verificando os dados brutos no link da api, escolhemos filtrar em 4 tópicos principais ['name','countries','population','continents']

df_paises = df[['name','capital','population','continents']].copy()

df_paises['capital'].apply(lambda x: x[0] if isinstance(x, list) else x)
df_paises['capital'].fillna('Sem Capital', inplace=True)
df_paises['capital'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_paises['population'].fillna(0, inplace = True)
df_paises['continents'].apply(lambda x: ', '.join(x) if isinstance(x, list) else x)
df_paises['name'] = df_paises['name'].apply(lambda x: x['common'] if isinstance(x, dict) else x)


#aqui vamos trocar o nome das colunas pra ficar mais fácil a leitura
df_paises = df_paises.rename(columns={'name': 'Nome do Pais', 'capital': 'Capital', 'population': 'Populacao', 'continents': 'Continentes'})

print('Nome das colunas atualizado')


df_paises.head()

Nome das colunas atualizado


,Nome do Pais,Capital,Populacao,Continentes
0,South Georgia,[King Edward Point],30,[Antarctica]
1,Grenada,[St. George's],112519,[North America]
2,Switzerland,[Bern],8654622,[Europe]
3,Sierra Leone,[Freetown],7976985,[Africa]
4,Hungary,[Budapest],9749763,[Europe]


In [160]:
df_paises= df_paises[df_paises['Populacao'] > 1_000_000]

df_paises.head()

,Nome do Pais,Capital,Populacao,Continentes
2,Switzerland,[Bern],8654622,[Europe]
3,Sierra Leone,[Freetown],7976985,[Africa]
4,Hungary,[Budapest],9749763,[Europe]
5,Taiwan,[Taipei],23503349,[Asia]
9,Ivory Coast,[Yamoussoukro],26378275,[Africa]


In [ ]:
# zerar a tabela
cur.execute("DROP TABLE IF EXISTS paises")
con.commit()
# Agora vamos criar a nossa tabela sql
cur.execute('''
CREATE TABLE IF NOT EXISTS paises (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    Nome_do_Pais TEXT,
    Capital TEXT,
    Populacao INTEGER,
    Continentes TEXT
)
''')
con.commit()


In [157]:
#Nisso vamos incluir nessa tabela os dados do nosso df_paises

# Inserir os dados do DataFrame na tabela
for index, row in df_paises.iterrows():
    cur.execute('''
    INSERT INTO paises (Nome_do_Pais, Capital, Populacao, Continentes)
    VALUES (?, ?, ?, ?)''', (row['Nome do Pais'], row['Capital'], row['Populacao'], row['Continentes']))
print("Dados inseridos com sucesso!")

# Salvar as alterações no banco de dados
con.commit()


Dados inseridos com sucesso!


In [ ]:
consulta = pd.read_sql_query("SELECT * FROM paises LIMIT 10;", con)
print("Dados Inseridos na Tabela:")
print(consulta)

Dados Inseridos na Tabela:
   id  Nome_do_Pais       Capital  Populacao Continentes
0   1   Switzerland          Bern    8654622      Europe
1   2  Sierra Leone      Freetown    7976985      Africa
2   3       Hungary      Budapest    9749763      Europe
3   4        Taiwan        Taipei   23503349        Asia
4   5   Ivory Coast  Yamoussoukro   26378275      Africa
5   6       Tunisia         Tunis   11818618      Africa
6   7         Italy          Rome   59554023      Europe
7   8         Benin    Porto-Novo   12123198      Africa
8   9     Indonesia       Jakarta  273523621        Asia
9  10          Laos     Vientiane    7275556        Asia
